In [1]:
from nfstream import NFStreamer, NFPlugin
import joblib
import pickle

In [2]:
filename = "RF_Model.pkl"
rfc_trained = joblib.load(filename)
rfc_trained

RandomForestClassifier(max_depth=110, max_features=2, min_samples_leaf=3,
                       min_samples_split=12)

In [3]:
class ContiniousPrediction(NFPlugin):
    def on_init(self, packet, flow):
        flow.udps.model_prediction = 0
    def on_expire(self, flow):
        to_predict = np.array([flow.protocol, flow.bidirectional_bytes,
                               flow.bidirectional_packets, flow.bidirectional_duration_ms,
                               flow.bidirectional_stddev_ps, flow.bidirectional_mean_ps,
                               flow.bidirectional_mean_piat_ms]).reshape((1,-1))
        flow.udps.model_prediction = self.my_model.predict(to_predict)
print('Function ModelPredict accepted')

Function ModelPredict accepted


In [ ]:
# Continious prediction
print("Continious Prediction :")
ml_streamer = NFStreamer(source="pcap.pcap",
                         statistical_analysis=True,
                         splt_analysis=0,
                         n_meters=0,
                         performance_report=0,
                         udps=ContiniousPrediction(my_model=rfc_trained))
for flow in ml_streamer:
    print(flow)